In [45]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


In [46]:
df = pd.read_csv('/content/data (1).csv', encoding='unicode_escape')

In [47]:
df.head()

,stn_code,sampling_date,state,location,agency,type,so2,no2,rspm,spm,location_monitoring_station,pm2_5,date
0,150.0,February - M021990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",4.8,17.4,NaN,NaN,NaN,NaN,1990-02-01
1,151.0,February - M021990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,3.1,7.0,NaN,NaN,NaN,NaN,1990-02-01
2,152.0,February - M021990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.2,28.5,NaN,NaN,NaN,NaN,1990-02-01
3,150.0,March - M031990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.3,14.7,NaN,NaN,NaN,NaN,1990-03-01
4,151.0,March - M031990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,4.7,7.5,NaN,NaN,NaN,NaN,1990-03-01


In [48]:
# Replace values in the 'type' column for consistency
df['type'].replace('Residential, Rural and other Areas','Residential',inplace = True)
df['type'].replace('Residential and others','Residential',inplace = True)
df['type'].replace('Industrial Areas','Industrial',inplace = True)
df['type'].replace('Industrial Area','Industrial',inplace = True)
df['type'].replace('Sensitive Area','Sensitive',inplace = True)
df['type'].replace('Sensitive Areas','Sensitive',inplace = True)

In [49]:
df['type'].unique()

array(['Residential', 'Industrial', nan, 'Sensitive', 'RIRUO'],
      dtype=object)

In [50]:
# Convert the 'date' column to datetime format and extract the 'year' column
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year

# Fill missing 'year' values using forward fill and convert to integer type
df['year'].fillna(method='ffill', inplace=True)
df['year'] = df['year'].astype(int)

df['year'].isnull().sum()  # Check for any remaining null values in the 'year' column

0

In [51]:
# Group 'air' DataFrame by 'location' and 'type' and create a new DataFrame with forward-filled 'rspm' and 'spm' values
df1 = dict(list(df.groupby(['location', 'type'])))
data = pd.DataFrame()
for i in df1:
    df2 = df1[i].sort_values('date')
    df2['rspm'].fillna(method='ffill', inplace=True)
    df2['spm'].fillna(method='ffill', inplace=True)
    data = pd.concat([data, df2])

In [52]:
# Group 'data' DataFrame by 'location' and 'type' and create a new DataFrame with backward-filled 'rspm' and 'spm' values
df1 = dict(list(data.groupby(['location', 'type'])))
data1 = pd.DataFrame()
for i in df1:
    df2 = df1[i].sort_values('date')
    df2['rspm'].fillna(method='bfill', inplace=True)
    df2['spm'].fillna(method='bfill', inplace=True)
    data1 = pd.concat([data1, df2])

In [53]:
data1.head()

,stn_code,sampling_date,state,location,agency,type,so2,no2,rspm,spm,location_monitoring_station,pm2_5,date,year
101624,SAMP,05-01-15,Gujarat,ANKLESHWAR,Gujarat State Pollution Control Board,RIRUO,13.0,20.0,82.0,NaN,"Panoli Ind.Asso. & Emergency Response Centre,P...",26.0,2015-01-05,2015
101541,SAMP,06-01-15,Gujarat,ANKLESHWAR,Gujarat State Pollution Control Board,RIRUO,13.0,20.0,91.0,NaN,"GIDC OFFICE TERRACE, GIDC ESTATE JHAGADIA,ANKL...",37.0,2015-01-06,2015
101625,SAMP,08-01-15,Gujarat,ANKLESHWAR,Gujarat State Pollution Control Board,RIRUO,14.0,21.0,70.0,NaN,"Panoli Ind.Asso. & Emergency Response Centre,P...",29.0,2015-01-08,2015
101542,SAMP,09-01-15,Gujarat,ANKLESHWAR,Gujarat State Pollution Control Board,RIRUO,14.0,21.0,78.0,NaN,"GIDC OFFICE TERRACE, GIDC ESTATE JHAGADIA,ANKL...",33.0,2015-01-09,2015
101626,SAMP,12-01-15,Gujarat,ANKLESHWAR,Gujarat State Pollution Control Board,RIRUO,14.0,21.0,82.0,NaN,"Panoli Ind.Asso. & Emergency Response Centre,P...",25.0,2015-01-12,2015


In [54]:
# Group 'data1' DataFrame by 'state' and 'type' and create a new DataFrame with median-filled 'rspm' and 'spm' values
df1 = dict(list(data1.groupby(['state', 'type'])))
data2 = pd.DataFrame()

# Iterate through groups and fill missing 'rspm' and 'spm' values with group-wise medians
for key in df1:
    df2 = df1[key]
    df2['rspm'].fillna(df2['rspm'].median(), inplace=True)
    df2['spm'].fillna(df2['spm'].median(), inplace=True)
    data2 = pd.concat([data2, df2])

In [55]:
data2

,stn_code,sampling_date,state,location,agency,type,so2,no2,rspm,spm,location_monitoring_station,pm2_5,date,year
1,151.0,February - M021990,Andhra Pradesh,Hyderabad,NaN,Industrial,3.1,7.0,130.3,82.000000,NaN,NaN,1990-02-01,1990
4,151.0,March - M031990,Andhra Pradesh,Hyderabad,NaN,Industrial,4.7,7.5,130.3,82.000000,NaN,NaN,1990-03-01,1990
7,151.0,April - M041990,Andhra Pradesh,Hyderabad,NaN,Industrial,4.7,8.7,130.3,82.000000,NaN,NaN,1990-04-01,1990
9,151.0,May - M051990,Andhra Pradesh,Hyderabad,NaN,Industrial,4.0,8.9,130.3,82.000000,NaN,NaN,1990-05-01,1990
12,151.0,June - M061990,Andhra Pradesh,Hyderabad,NaN,Industrial,5.6,11.8,130.3,82.000000,NaN,NaN,1990-06-01,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434695,650,12-12-15,West Bengal,South Suburban,West Bengal State Pollution Control Board,Residential,2.0,44.0,93.0,577.666667,"Baruipur, South Suburban",NaN,2015-12-12,2015
434696,650,14-12-15,West Bengal,South Suburban,West Bengal State Pollution Control Board,Residential,2.0,47.0,145.0,577.666667,"Baruipur, South Suburban",NaN,2015-12-14,2015
434697,650,18-12-15,West Bengal,South Suburban,West Bengal State Pollution Control Board,Residential,4.0,55.0,208.0,577.666667,"Baruipur, South Suburban",NaN,2015-12-18,2015
434698,650,20-12-15,West Bengal,South Suburban,West Bengal State Pollution Control Board,Residential,3.0,49.0,206.0,577.666667,"Baruipur, South Suburban",NaN,2015-12-20,2015


In [56]:
# Group 'data2' DataFrame by 'type' and create a new DataFrame with median-filled 'rspm' and 'spm' values
df1 = dict(list(data2.groupby('type')))
data3 = pd.DataFrame()
for i in df1:
    df2 = df1[i]
    df2['rspm'].fillna(df2['rspm'].median(), inplace=True)
    df2['spm'].fillna(df2['spm'].median(), inplace=True)
    data3 = pd.concat([data3, df2])

In [57]:
data3

,stn_code,sampling_date,state,location,agency,type,so2,no2,rspm,spm,location_monitoring_station,pm2_5,date,year
1,151.0,February - M021990,Andhra Pradesh,Hyderabad,NaN,Industrial,3.1,7.0,130.3,82.0,NaN,NaN,1990-02-01,1990
4,151.0,March - M031990,Andhra Pradesh,Hyderabad,NaN,Industrial,4.7,7.5,130.3,82.0,NaN,NaN,1990-03-01,1990
7,151.0,April - M041990,Andhra Pradesh,Hyderabad,NaN,Industrial,4.7,8.7,130.3,82.0,NaN,NaN,1990-04-01,1990
9,151.0,May - M051990,Andhra Pradesh,Hyderabad,NaN,Industrial,4.0,8.9,130.3,82.0,NaN,NaN,1990-05-01,1990
12,151.0,June - M061990,Andhra Pradesh,Hyderabad,NaN,Industrial,5.6,11.8,130.3,82.0,NaN,NaN,1990-06-01,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406659,1,29-12-15,Uttar Pradesh,Agra,Central Pollution Control Board,Sensitive,2.0,27.0,211.0,448.0,"Taj Mahal, Agra",NaN,2015-12-29,2015
406660,1,30-12-15,Uttar Pradesh,Agra,Central Pollution Control Board,Sensitive,2.0,34.0,352.0,448.0,"Taj Mahal, Agra",NaN,2015-12-30,2015
408838,415,30-12-15,Uttar Pradesh,Agra,Central Pollution Control Board,Sensitive,4.0,36.0,427.0,448.0,"DIC Nunhai, Agra",NaN,2015-12-30,2015
406661,1,31-12-15,Uttar Pradesh,Agra,Central Pollution Control Board,Sensitive,2.0,31.0,363.0,448.0,"Taj Mahal, Agra",NaN,2015-12-31,2015


# **Data Saving**

In [58]:
# Reset index and drop unnecessary columns from the 'data3' DataFrame
data3.reset_index(inplace=True)
data3.drop(columns=['index', 'stn_code', 'sampling_date', 'agency', 'location_monitoring_station'], inplace=True)
data3.head()

,state,location,type,so2,no2,rspm,spm,pm2_5,date,year
0,Andhra Pradesh,Hyderabad,Industrial,3.1,7.0,130.3,82.0,NaN,1990-02-01,1990
1,Andhra Pradesh,Hyderabad,Industrial,4.7,7.5,130.3,82.0,NaN,1990-03-01,1990
2,Andhra Pradesh,Hyderabad,Industrial,4.7,8.7,130.3,82.0,NaN,1990-04-01,1990
3,Andhra Pradesh,Hyderabad,Industrial,4.0,8.9,130.3,82.0,NaN,1990-05-01,1990
4,Andhra Pradesh,Hyderabad,Industrial,5.6,11.8,130.3,82.0,NaN,1990-06-01,1990


Here we check for missing values in the '`data3`' DataFrame to ensure that the dataset is free of any remaining null values after the preprocessing steps.

In [59]:
data3.isnull().sum()

state            0
location         0
type             0
so2          34188
no2          15848
rspm             0
spm           1304
pm2_5       421035
date             4
year             0
dtype: int64

In [60]:
data3.to_csv('air_quality_cleaned_data.csv', index=False)